In [ ]:
#!pip uninstall numpy -y
#!pip install numpy==1.23.5  # Example: Install a specific version
#!pip install gensim

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
from google.colab import files
f=files.upload()

Saving EgyDrugs_Dataset222.csv to EgyDrugs_Dataset222.csv


In [4]:
df = pd.read_csv(r'EgyDrugs_Dataset222.csv')
df.head()

,id,name,arabic,oldprice,price,active,company,description,units,dosage_form,barcode,imported,Date_updated,All Active Ingredients,Primary Active Ingredient,side_effects,value
0,2,1 2 3 (one two three) 20 f.c.tabs.,ون تو ثري 20 قرص,24.0,40.0,pseudoephedrine+paracetamol+chlorpheniramine,hikma pharmaceuticals,cold drugs,2,tablet,NaN,local,1.726670e+12,pseudoephedrine + paracetamol + chlorpheniramine,pseudoephedrine,"liver damage (high doses), nausea, rash, drows...",20 f.c.tabs
1,1,1 2 3 (one two three) syrup 120 ml,ون تو ثري شراب 120 مل,19.5,32.0,pseudoephedrine+paracetamol+chlorpheniramine,hikma pharmaceuticals,cold drugs,1,syrup,6221000000003,local,1.720117e+12,pseudoephedrine + paracetamol + chlorpheniramine,pseudoephedrine,"liver damage (high doses), nausea, rash, drows...",120 ml
2,13245,1 2 3 extra 20 f.c. tab.,ون تو ثري اكسترا 20 قرص,33.0,50.0,chlorpheniramine+paracetamol(acetaminophen)+ps...,hikma pharmaceuticals,cold drugs,2,tablet,6221000010828,local,1.724533e+12,chlorpheniramine + paracetamol(acetaminophen) ...,chlorpheniramine,"drowsiness, dry mouth, dizziness, blurred visi...",NaN
3,13247,4 wet intimate gel 100 ml,فور ويت انتيميت جل 100 مل,NaN,40.0,NaN,brother pharma + turkuaz health care,lubricant,1,gel,NaN,local,1.659994e+12,NaN,NaN,NaN,100 ml
4,3,5-fluorouracil-ebewe 250mg/5ml i.v. vial,فلورويوراسيل 250مجم/10مل 5 امبولات وريد,NaN,50.0,fluorouracil,shanghai xudong haipu pharmaceutical + acdima ...,antineoplastic,1,vial,9088881320685,imported,1.670251e+12,fluorouracil,fluorouracil,"nausea, vomiting, diarrhea, skin irritation, b...",250mg + 5ml


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20628 entries, 0 to 20627
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         20628 non-null  int64  
 1   name                       20628 non-null  object 
 2   arabic                     20627 non-null  object 
 3   oldprice                   6956 non-null   float64
 4   price                      20626 non-null  float64
 5   active                     19095 non-null  object 
 6   company                    20628 non-null  object 
 7   description                20613 non-null  object 
 8   units                      20628 non-null  int64  
 9   dosage_form                20626 non-null  object 
 10  barcode                    7761 non-null   object 
 11  imported                   20628 non-null  object 
 12  Date_updated               18544 non-null  float64
 13  All Active Ingredients     19094 non-null  obj

In [6]:
df = df[df['All Active Ingredients'].notnull()]

In [7]:
x = df['description'].unique()
for i in x:
  print(i)

cold drugs
antineoplastic
vitamin a
immunoglobulin
soothing agent
soothing topical
vitamin c
multivitamins
antiseptic
omega 3
whitening topical
vaccine
laxative
psychiatric antipsychotics
antipyretic analgesic
analgesic antipyretic headache
oral care
vitamin d
immunosuppressant
immunosuppressants
antioxidant
psychiatric antidepressants
analgesics opioid
chocolate-flavored candy
hair care
diaper rash
moisturizing topicals
unknown
antihypertensive ace
massage cream
mucolytic
vitamin b
granulocyte colony-stimulating
leukotriene d4
androgen
nsaid acetic acid
eczema
nsaid analgesic
antipyretic
diuretic carbonic anhydrase
analgesic antipyretic
analgesic antipyretic nsaid
glucocorticoid
vaginal care
cold products
menstrual pain
antidiarrheal
mucolytic soothing
cough syrup
antiparkinson agents
anthelmintic
antiviral nucleoside
antacid
skin care acne
psoriasis
anti-viral nucleoside
osteoporosis bisphosphonates
anti-acne
skin care anti-acne
antibiotic quinolone
anti-acne antibiotic macrolide
ant

In [8]:
df['company'].nunique()

3962

In [9]:
df['dosage_form'].unique()

array(['tablet', 'syrup', 'vial', 'capsule', 'cream', 'sachet', 'spray',
       'oral drops', 'suppository', 'tablets', 'syringe', 'piece',
       'eye drops', 'gel', 'effervescent', 'solution', 'suspension',
       'vaginal douche', 'lotion', 'bottle', 'paint', 'facial wash',
       'hair oil', 'pen', 'ampoule', 'eye ointment', 'ointment',
       'nasal drops', 'mouth wash', 'shampoo', 'ear drops', 'mouth drops',
       'powder', 'power ', 'cream ', 'lozenges', 'soap', 'tablet ',
       'suspension ', 'capsule ', 'dro', 'patch', 'tabs.', 'film',
       'serum', 'oil', 'foam', nan, 'conditioner', 'caps'], dtype=object)

In [10]:
df['dosage_form'] = df['dosage_form'].str.strip().replace({'tablet ': 'tablet', 'tablets': 'tablet',
                                                           'capsule ': 'capsule', 'caps': 'capsule',
                                                           'cream ': 'cream', 'suspension ': 'suspension',
                                                           'power ': 'powder', 'dro': 'ear drops',
                                                           'tabs.': 'tablet'})


In [11]:
df.loc[(df['dosage_form'].isnull()) & (df['name'] == 'icoface 60 gm sunscreen cream'), 'dosage_form'] = 'cream'

In [12]:
df.loc[df['dosage_form'].isnull(), 'dosage_form'] = 'syrup'

In [13]:
for col in df.select_dtypes(include='object').columns:
  df[col] = df[col].astype(str).apply(lambda x: x.lower() if x != "nan" else None)

In [14]:
df.head()

,id,name,arabic,oldprice,price,active,company,description,units,dosage_form,barcode,imported,Date_updated,All Active Ingredients,Primary Active Ingredient,side_effects,value
0,2,1 2 3 (one two three) 20 f.c.tabs.,ون تو ثري 20 قرص,24.0,40.0,pseudoephedrine+paracetamol+chlorpheniramine,hikma pharmaceuticals,cold drugs,2,tablet,None,local,1.726670e+12,pseudoephedrine + paracetamol + chlorpheniramine,pseudoephedrine,"liver damage (high doses), nausea, rash, drows...",20 f.c.tabs
1,1,1 2 3 (one two three) syrup 120 ml,ون تو ثري شراب 120 مل,19.5,32.0,pseudoephedrine+paracetamol+chlorpheniramine,hikma pharmaceuticals,cold drugs,1,syrup,6221000000003,local,1.720117e+12,pseudoephedrine + paracetamol + chlorpheniramine,pseudoephedrine,"liver damage (high doses), nausea, rash, drows...",120 ml
2,13245,1 2 3 extra 20 f.c. tab.,ون تو ثري اكسترا 20 قرص,33.0,50.0,chlorpheniramine+paracetamol(acetaminophen)+ps...,hikma pharmaceuticals,cold drugs,2,tablet,6221000010828,local,1.724533e+12,chlorpheniramine + paracetamol(acetaminophen) ...,chlorpheniramine,"drowsiness, dry mouth, dizziness, blurred visi...",None
4,3,5-fluorouracil-ebewe 250mg/5ml i.v. vial,فلورويوراسيل 250مجم/10مل 5 امبولات وريد,NaN,50.0,fluorouracil,shanghai xudong haipu pharmaceutical + acdima ...,antineoplastic,1,vial,9088881320685,imported,1.670251e+12,fluorouracil,fluorouracil,"nausea, vomiting, diarrhea, skin irritation, b...",250mg + 5ml
5,13249,5fu 250mg/5ml vial,5-فلورويوراسيل 250مجم/5مل فيال,NaN,9.0,fluorouracil,choongwae pharma-korea + general technology-egypt,antineoplastic,1,vial,None,local,1.657574e+12,fluorouracil,fluorouracil,"nausea, vomiting, diarrhea, skin irritation, b...",250mg + 5ml


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19094 entries, 0 to 20625
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         19094 non-null  int64  
 1   name                       19094 non-null  object 
 2   arabic                     19093 non-null  object 
 3   oldprice                   6583 non-null   float64
 4   price                      19092 non-null  float64
 5   active                     19094 non-null  object 
 6   company                    19094 non-null  object 
 7   description                19080 non-null  object 
 8   units                      19094 non-null  int64  
 9   dosage_form                19094 non-null  object 
 10  barcode                    7541 non-null   object 
 11  imported                   19094 non-null  object 
 12  Date_updated               17150 non-null  float64
 13  All Active Ingredients     19094 non-null  object 


In [16]:
df[df['price'].isnull()]

,id,name,arabic,oldprice,price,active,company,description,units,dosage_form,barcode,imported,Date_updated,All Active Ingredients,Primary Active Ingredient,side_effects,value
6515,30497,esmopral 40 mg 14 tablet,ايزموبرال 40مجم 14 قرص,NaN,NaN,esmoprazole,egyptian group for pharmaceutical industries +...,proton pump,2,tablet,None,local,NaN,esmoprazole,esmoprazole,None,40 mg + 14 tab
6886,30525,fast slim capsule,فاست سليم كبسول,NaN,NaN,garcinia extract+green tea extract+green coffe...,tahoun pharma,weigh loss,1,capsule,None,local,NaN,garcinia extract + green tea extract + green c...,garcinia extract,None,None


In [17]:
df=df[~df['price'].isnull()]

In [18]:
df['price'].isnull().sum()

np.int64(0)

In [19]:
df.rename(columns={'All Active Ingredients': 'entire_comp','Primary Active Ingredient':'primary_comp'}, inplace=True)

In [20]:
df.columns

Index(['id', 'name', 'arabic', 'oldprice', 'price', 'active', 'company',
       'description', 'units', 'dosage_form', 'barcode', 'imported',
       'Date_updated', 'entire_comp', 'primary_comp', 'side_effects', 'value'],
      dtype='object')

In [21]:
df.to_excel('EgyptianDrugs00.xlsx', index=False)

In [22]:
df.drop(columns=['id','oldprice','active','Date_updated','imported','arabic','units'],inplace=True)

In [23]:
df.loc[df['name'] == '1 2 3 (one two three) 20 f.c.tabs.', 'name'] = '1 2 3 (one two three) 20 f.c. tab'

In [24]:
df.loc[df['name'] == '1 2 3 (one two three) 20 f.c. tab', 'name'] = 'one two three 20 f.c. tab'
df.loc[df['name'] == '1 2 3 (one two three) syrup 120 ml', 'name'] = 'one two three 20 f.c. tab'
df.loc[df['name'] == '1 2 3 extra 20 f.c. tab.', 'name'] = 'one two three extra 20 f.c. tab'

In [25]:
df.head(10)

,name,price,company,description,dosage_form,barcode,entire_comp,primary_comp,side_effects,value
0,one two three 20 f.c. tab,40.0,hikma pharmaceuticals,cold drugs,tablet,None,pseudoephedrine + paracetamol + chlorpheniramine,pseudoephedrine,"liver damage (high doses), nausea, rash, drows...",20 f.c.tabs
1,one two three 20 f.c. tab,32.0,hikma pharmaceuticals,cold drugs,syrup,6221000000003,pseudoephedrine + paracetamol + chlorpheniramine,pseudoephedrine,"liver damage (high doses), nausea, rash, drows...",120 ml
2,one two three extra 20 f.c. tab,50.0,hikma pharmaceuticals,cold drugs,tablet,6221000010828,chlorpheniramine + paracetamol(acetaminophen) ...,chlorpheniramine,"drowsiness, dry mouth, dizziness, blurred visi...",None
4,5-fluorouracil-ebewe 250mg/5ml i.v. vial,50.0,shanghai xudong haipu pharmaceutical + acdima ...,antineoplastic,vial,9088881320685,fluorouracil,fluorouracil,"nausea, vomiting, diarrhea, skin irritation, b...",250mg + 5ml
5,5fu 250mg/5ml vial,9.0,choongwae pharma-korea + general technology-egypt,antineoplastic,vial,None,fluorouracil,fluorouracil,"nausea, vomiting, diarrhea, skin irritation, b...",250mg + 5ml
6,a-viton 50.000 i.u. 20 caps.,19.0,kahira,vitamin a,capsule,6221068101001,vitamin a,vitamin a,None,20 caps
8,a.t.g fresenius 20mg/ml conc. vial 5 ml,1100.0,fresenius kabi,immunoglobulin,vial,None,anti-t lymphocyte(anti-thymocyte) immunoglobul...,anti-t lymphocyte(anti-thymocyte) immunoglobul...,None,20mg + 5 ml
9,a1 cream 100 gm,45.0,egd + egpi,soothing agent,cream,6223003930479,castor oil + zinc oxide + arachis oil + bees wax,castor oil,None,1 cream + 100 g
10,a1 cream 50 gm,30.0,egd + egpi,soothing topical,cream,6223003932084,castor oil + zinc oxide + arachis oil + bees wax,castor oil,None,1 cream + 50 g
15,abc immune 10 sachets,40.0,aug pharma,vitamin c,sachet,None,vitamin c + zinc,vitamin c,None,10 sachets


In [26]:
df=df[~df['description'].isnull()]

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19078 entries, 0 to 20625
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          19078 non-null  object 
 1   price         19078 non-null  float64
 2   company       19078 non-null  object 
 3   description   19078 non-null  object 
 4   dosage_form   19078 non-null  object 
 5   barcode       7538 non-null   object 
 6   entire_comp   19078 non-null  object 
 7   primary_comp  19078 non-null  object 
 8   side_effects  178 non-null    object 
 9   value         17874 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.6+ MB


In [28]:
import re

columns_to_process = ["name", "company", "dosage_form", "description", "entire_comp", "primary_comp"]
df_selected = df[columns_to_process].copy()

def clean_text(text, keep_numbers=False):
    if pd.isna(text):
        return ""
    text = text.lower()
    if not keep_numbers:
        text = re.sub(r"[^a-zA-Z\s\+/-]", "", text)
    else:
        text = re.sub(r"[^a-zA-Z0-9\s\+/-]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def tokenize_name(text):
    text = text.replace("(", "").replace(")", "")
    text = re.sub(r"(\d+)(mg|ml|g|kg|mcg|ug|tab|caps|vial|dose|tabs)", r"\1 \2", text)
    text = re.sub(r"(\d+)/(\d+)(ml|mg|g|kg)", r"\1 \2 \3", text)
    text = re.sub(r"\b(i\.v\.|iv)\b", "iv", text)
    text = re.sub(r"\b(sol\.|soln\.)\b", "solution", text)
    text = re.sub(r"\b(susp\.|suspension)\b", "suspension", text)
    text = re.sub(r"\b(f\.c\.|fc|film coated)\b", "film-coated", text)
    text = re.sub(r"\b(ext\.|extended release)\b", "extended-release", text)
    text = re.sub(r"\b(cr\.|controlled release)\b", "controlled-release", text)
    text = text.replace(" - ", "-")
    return text.split()
# Apply cleaning function to all selected columns
for col in columns_to_process:
    keep_numbers = True if col == "name" else False
    df_selected[col] = df_selected[col].apply(lambda x: clean_text(x, keep_numbers))

# Tokenize the name column with improved splitting
df_selected["tokenized_name"] = df_selected["name"].apply(tokenize_name)

# Tokenize other columns for Word2Vec
df_selected["tokenized_company"] = df_selected["company"].apply(lambda x: x.split())
df_selected["tokenized_dosage_form"] = df_selected["dosage_form"].apply(lambda x: x.split())
df_selected["tokenized_description"] = df_selected["description"].apply(lambda x: x.split())

# Split active ingredients by '+' to get individual components
df_selected["tokenized_all_active_ingredients"] = df_selected["entire_comp"].apply(lambda x: x.split("+"))
df_selected["tokenized_primary_active_ingredients"] = df_selected["primary_comp"].apply(lambda x: x.split())


In [29]:
df_selected.head()

,name,company,dosage_form,description,entire_comp,primary_comp,tokenized_name,tokenized_company,tokenized_dosage_form,tokenized_description,tokenized_all_active_ingredients,tokenized_primary_active_ingredients
0,one two three 20 fc tab,hikma pharmaceuticals,tablet,cold drugs,pseudoephedrine + paracetamol + chlorpheniramine,pseudoephedrine,"[one, two, three, 20, film-coated, tab]","[hikma, pharmaceuticals]",[tablet],"[cold, drugs]","[pseudoephedrine , paracetamol , chlorphenir...",[pseudoephedrine]
1,one two three 20 fc tab,hikma pharmaceuticals,syrup,cold drugs,pseudoephedrine + paracetamol + chlorpheniramine,pseudoephedrine,"[one, two, three, 20, film-coated, tab]","[hikma, pharmaceuticals]",[syrup],"[cold, drugs]","[pseudoephedrine , paracetamol , chlorphenir...",[pseudoephedrine]
2,one two three extra 20 fc tab,hikma pharmaceuticals,tablet,cold drugs,chlorpheniramine + paracetamolacetaminophen + ...,chlorpheniramine,"[one, two, three, extra, 20, film-coated, tab]","[hikma, pharmaceuticals]",[tablet],"[cold, drugs]","[chlorpheniramine , paracetamolacetaminophen ...",[chlorpheniramine]
4,5-fluorouracil-ebewe 250mg/5ml iv vial,shanghai xudong haipu pharmaceutical + acdima ...,vial,antineoplastic,fluorouracil,fluorouracil,"[5-fluorouracil-ebewe, 250, mg/5, ml, iv, vial]","[shanghai, xudong, haipu, pharmaceutical, +, a...",[vial],[antineoplastic],[fluorouracil],[fluorouracil]
5,5fu 250mg/5ml vial,choongwae pharma-korea + general technology-egypt,vial,antineoplastic,fluorouracil,fluorouracil,"[5fu, 250, mg/5, ml, vial]","[choongwae, pharma-korea, +, general, technolo...",[vial],[antineoplastic],[fluorouracil],[fluorouracil]


In [30]:
df.to_csv('EgyptianDrugsBackEnd.csv', index=False)

In [32]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313504 sha256=bfb04e813e8d90f327ce5fd21342d47840a45f0377f20d4054c66635a2b00ff0
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [33]:
import fasttext

tokenized_columns = ['tokenized_name', 'tokenized_company', 'tokenized_dosage_form',
                     'tokenized_description', 'tokenized_all_active_ingredients']

training_data = df_selected[tokenized_columns].apply(lambda x: ' '.join([' '.join(map(str, tokens)) for tokens in x]), axis=1).tolist()

with open('fasttext_training_data.txt', 'w') as f:
    for line in training_data:
        f.write(f"{line}\n")

model = fasttext.train_unsupervised('fasttext_training_data.txt', model='skipgram')

model.save_model("fasttext_model.bin")


In [34]:
weights = {
    "name": 1.0,
    "company": 0.2,
    "dosage_form": 0.7,
    "description": 0.7,
    "entire_comp": 5.0
}

In [35]:
def get_fasttext_embedding(tokens, model):
    embeddings = [model.get_word_vector(token) for token in tokens if token in model]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.get_dimension())  # Return zero vector if no tokens are in the model

def get_weighted_embedding(row, model, weights):
    # Extract tokenized columns and calculate embeddings
    name_embedding = get_fasttext_embedding(row['tokenized_name'], model) * weights['name']
    company_embedding = get_fasttext_embedding(row['tokenized_company'], model) * weights['company']
    dosage_form_embedding = get_fasttext_embedding(row['tokenized_dosage_form'], model) * weights['dosage_form']
    description_embedding = get_fasttext_embedding(row['tokenized_description'], model) * weights['description']
    entire_comp_embedding = get_fasttext_embedding(row['tokenized_all_active_ingredients'], model) * weights['entire_comp']

    weighted_embedding = name_embedding + company_embedding + dosage_form_embedding + description_embedding + entire_comp_embedding

    return weighted_embedding


In [36]:
# Generate weighted embeddings for all drugs
drug_embeddings = {}

for index, row in df_selected.iterrows():
    weighted_embedding = get_weighted_embedding(row, model, weights)
    drug_embeddings[index] = weighted_embedding


In [ ]:
# User input drug name
input_drug_name = input("Enter the drug name: ").strip().lower()
# Step 1: Find the drug in the dataset
matched_drug = df_selected[df_selected['name'].str.lower() == input_drug_name.lower()]

if matched_drug.empty:
    print("Drug not found in the dataset. Please enter a valid drug name.")
else:
    # Step 2: Extract details from the dataset
    matched_row = matched_drug.iloc[0]  # Get the first matching row

    # Step 3: Generate weighted embedding using all attributes
    input_weighted_embedding = get_weighted_embedding(matched_row, model, weights)

    # Step 4: Compute similarity with all drugs
    all_embeddings = np.array(list(drug_embeddings.values()))
    similarities = cosine_similarity([input_weighted_embedding], all_embeddings)

    top_n = 10
    top_indices = similarities[0].argsort()[-top_n:][::-1]

    similar_drug_names = df_selected.iloc[top_indices]['name']
    similar_drug_active = df_selected.iloc[top_indices]['entire_comp']

    for name, active in zip(similar_drug_names, similar_drug_active):
        print(f"name: {name} -- active ingredient: {active}")


In [ ]:
df_selected[df_selected['name'].str.contains('panadol')]

In [ ]:
df_selected.to_csv("Final_Drugs.csv", index=False)

In [ ]:
model = fasttext.train_unsupervised('fasttext_training_data.txt', model='skipgram',dim = 100)
model.save_model("fasttext_model.ftz")

print("✅ Model retrained with smaller vector size! Check the new file size.")